In [1]:
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()
mongo='127.0.0.1'

try:
    # Connect to MongoDB
    myclient = MongoClient(
                        "mongodb://"+mongo+":27017/",  
                        username='admin',
                        password='admin') #Mongo URI format
    db=myclient['reddit']
    print("Connected to MongoDB successfully!")
except Exception as e:
    print("An error occurred while connecting to MongoDB:", e)

Connected to MongoDB successfully!


In [3]:
#import
posts_registered=[]
query=db.reddit_posts.find({},{'_id':0})
for el in query:
    posts_registered.append(el)
posts_registered_df=pd.DataFrame(posts_registered)
posts_registered_df.head()

OperationFailure: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}

In [6]:
posts_registered_df['prompt']=posts_registered_df['title']+'\n'+posts_registered_df['self_text']
posts_registered_df.head()

,id,title,author,score,num_comments,upvote_ratio,url,subreddit,created_at,self_text,searchQuery,prompt
0,1et3kj0,Diagnosed with Inattentive ADHD at 31. Explain...,amadnomad,392,107,0.98,https://www.reddit.com/r/ADHD/comments/1et3kj0...,ADHD,1.723749e+09,Please go out and get tested if you are still ...,adhd,Diagnosed with Inattentive ADHD at 31. Explain...
1,1gk5ftv,Children with higher IQ scores were diagnosed ...,Pretend_Voice_3140,4499,466,0.99,https://www.reddit.com/r/ADHD/comments/1gk5ftv...,ADHD,1.730809e+09,A study was published in the [British Journal ...,adhd,Children with higher IQ scores were diagnosed ...
2,1f0k9en,Reminder: If you made it to adulthood with lat...,Hipster-Deuxbag,7372,535,0.99,https://www.reddit.com/r/ADHD/comments/1f0k9en...,ADHD,1.724547e+09,"We all know the statistics: 20,000 behavioral ...",adhd,Reminder: If you made it to adulthood with lat...
3,1g7tbb2,What is the most adhd thing you have ever adhd’ed,FamiliarRadio9275,2102,1022,0.99,https://www.reddit.com/r/ADHD/comments/1g7tbb2...,ADHD,1.729409e+09,I laugh so hard when I look back at this memor...,adhd,What is the most adhd thing you have ever adhd...
4,1dy8uqs,I’m angry that no one recognized that I had AD...,thecynicalone26,3345,598,0.99,https://www.reddit.com/r/ADHD/comments/1dy8uqs...,ADHD,1.720445e+09,"I just got diagnosed, and I’m 39. My entire l...",adhd,I’m angry that no one recognized that I had AD...


In [7]:
posts_registered_df['prompt'].iloc[0]

"Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.\nPlease go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. \n\n  \nAgain, please go get tested if you suspect anything."

Analyze the following Reddit post and extract the following features:
- Sentiment: Is the sentiment of the post positive, negative, or neutral?
- Topic: What are the main topic discussed in the post (do not put adhd I want someting more specific?
- Personal experience shared? (Yes/No):
- Mention of solutions? (Yes/No):
-self diagnosis?
-gendre of person?
-self mediaction? 
→ If there is no clear mention put null and answer only in yes no to explanation required
Post: "Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.\nPlease go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. \n\n  \nAgain, please go get tested if you suspect anything."

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load your model
model_path = "./Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


# Set pad token ID to eos token ID if pad token ID is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id



In [9]:

# Function to analyze a post and extract variables
def extract_features(post_text):
    prompt = f"""
        

        Analyze the following Reddit post and extract the following features:
- Sentiment: Is the sentiment of the post positive, negative, or neutral?
- Topic: What are the main topic discussed in the post (do not put adhd I want someting more specific?
- Personal experience shared? (Yes/No):
- Mention of solutions? (Yes/No):
-self diagnosis?
-gendre of person?
-self mediaction? 
→ If there is no clear mention put null and answer only in yes no to explanation required
Post: "Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.\nPlease go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. \n\n  \nAgain, please go get tested if you suspect anything."
        """


    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(
    input_ids=inputs["input_ids"], 
    attention_mask=inputs["attention_mask"], 
    max_length=500,  # Adjust max length to prevent truncation
    num_beams=5,  # Use beam search for better responses
    temperature=0.7,  # Lower for more focused answers
    top_p=0.9,  # Nucleus sampling
    pad_token_id=tokenizer.eos_token_id
)

    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    print(f"Token length: {input_ids.shape[1]}")


    # Decode and return response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [ ]:

# Apply the feature extraction function to all posts
df["features"] = df["prompt"].apply(extract_features)

# Optional: Parse the features into structured columns (next step)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_path = "./Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Assign pad_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the Reddit post
post_text = "I'm struggling with ADHD. It's hard to focus on tasks and get things done. Any advice would help!"

# Create the prompt
prompt = f"""
Question: What is the capital of France?
Provide the answer in one word only:

"""

# Tokenize the input with padding
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# Generate response
output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=300,
    num_beams=5,
    temperature=0.7,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.pad_token_id
)

# Decode and print the response
response = tokenizer.decode(output[0], skip_special_tokens=True)
response = response[len(prompt):].strip()  # Remove the prompt part
print(response)

print(response)


/home/ysidhom/Documents/5IF/adhd_project/mental_health_disorders_analysis/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer: PARIS

Explanation: None
Answer: PARIS

Explanation: None


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_path = "./Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Assign pad_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the prompt
prompt = """
Analyze the following Reddit post and extract the following features:
- Sentiment: Is the sentiment of the post positive, negative, or neutral?
- Topic: What are the main topic discussed in the post (do not put adhd I want someting more specific?
- Personal experience shared? (Yes/No):
- Mention of solutions? (Yes/No):
-self diagnosis?
-gendre of person?
-self mediaction? 
→ If there is no clear mention put null and answer only in yes no to explanation required
Post: "Diagnosed with Inattentive ADHD at 31. Explains so many things from my childhood.\nPlease go out and get tested if you are still on the fence. I always assumed ADHD was only hyperactive. A lot of concerns about day dreaming, zoning out and inattentiveness came into play during my consult. I didn't even consider my lack of sleep being tied to ADHD. But now that I have a diagnoses, it explains quite a bit from my past. I wasn't just lazy and disorganized. \n\n  \nAgain, please go get tested if you suspect anything."
"""
# Tokenize the input with padding
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

# Generate response
output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=250,
    num_beams=5,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.pad_token_id
)

# Decode the response and remove the prompt
response = tokenizer.decode(output[0], skip_special_tokens=True)
response = response[len(prompt):].strip()  # Extract only the generated response
print(response)



Topic: 
Personal Experience Shared: Yes
Mention of Solutions: No
Self Diagnosis


### Fine Tunning the Model 


In [2]:
from datasets import load_dataset

# Load Reddit dataset (example: pushshift dataset)
dataset = load_dataset("jsfactory/mental_health_reddit_posts")

In [3]:
dataset# Split into train and eval (80% for training, 20% for evaluation)
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']


In [4]:
def preprocess_data(example):
    return tokenizer(
        example["body"],  # Replace "body" with the field containing the Reddit post
        truncation=True,
        padding="max_length",
        max_length=512,  # Adjust based on LLaMA's context length
    )

# Map the preprocessing function to the dataset
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token if not defined


train_dataset = train_dataset.map(preprocess_data, batched=True)
eval_dataset = eval_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForCausalLM

# Load your LLaMA model
model = AutoModelForCausalLM.from_pretrained(model_path)
model.resize_token_embeddings(len(tokenizer))  # Adjust token embeddings if vocabulary changes


Embedding(128256, 2048)

In [6]:
# Define training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine_tuned_llama",  # Output directory for model checkpoints
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_strategy="epoch",  # Save model after each epoch
    save_total_limit=2,  # Limit to two model checkpoints
)


In [ ]:

# Initialize the Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding

# Create a DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Pass the eval dataset here
    data_collator=data_collator,
)

# Start training
trainer.train()


  0%|          | 0/7200 [00:00<?, ?it/s]